In [1]:
dl_project_folder_location = "/content/drive/MyDrive/Colman/DL Project"

# Installs

In [2]:
!pip install colabcode
!pip install fastapi
!pip install aiofiles
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of nbclassic to determine which version is compatible with other requirements. This could take

# Prediction

## Connect drive for model files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Prediction Code

In [4]:
import multiprocessing as mp
import pathlib
import random
from pathlib import Path

import cv2
import dlib
import keras
import numpy as np
import tensorflow as tf
from PIL import Image

In [5]:
model = keras.models.load_model(f'{dl_project_folder_location}/Model Saves/50_words_augmentation_callbacks_79acc_20230602')
root_path = Path.cwd()

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [6]:
def load_file(file_path):
    data = np.load(file_path)
    return data['data']

def expand_image_channels(image):
    image_arr = np.array(image)
    image_arr = np.expand_dims(image_arr, 1)
    return image_arr

class DataGenerator:

    def __init__(self, path, set_type):
        """ Returns a set of frames with their associated label.

          Args:
            path: file paths (directory path).
            set_type: type of set to create - train, val or test.
        """
        self.path = path
        self.set_type = set_type
        self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir() and not p.name.startswith('.')))
        self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

    def get_files_and_class_names(self):
        data_paths = sorted(list(self.path.glob(f'**/{self.set_type}/*.npz')))
        classes = [p.parent.parent.name for p in data_paths]
        return data_paths, classes

    def __call__(self):
        data_paths, classes = self.get_files_and_class_names()
        pairs = list(zip(data_paths, classes))

        if self.set_type == 'train':
            random.shuffle(pairs)

        for path, name in pairs:
            video_frames = load_file(path)
            video_frames = expand_image_channels(video_frames)
            label = self.class_ids_for_name[name] # Encode labels
            yield video_frames, label

In [7]:
from collections import defaultdict

label_dict = {
  'ACTION': 0,
  'ALWAYS': 1,
  'BECAUSE': 2,
  'BECOME': 3,
  'BELIEVE': 4,
  'CALLED': 5,
  'CHANGE': 6,
  'CLOSE': 7,
  'DECIDED': 8,
  'DIFFERENT': 9,
  'EVERYONE': 10,
  'FOOTBALL': 11,
  'FOUND': 12,
  'GREAT': 13,
  'HAPPENED': 14,
  'HOSPITAL': 15,
  'INFORMATION': 16,
  'INSIDE': 17,
  'JUSTICE': 18,
  'LITTLE': 19,
  'MAYBE': 20,
  'MISSING': 21,
  'MONEY': 22,
  'NUMBER': 23,
  'OFFICE': 24,
  'OTHER': 25,
  'PARTY': 26,
  'PEOPLE': 27,
  'PLACES': 28,
  'PRESS': 29,
  'RESULT': 30,
  'SCHOOL': 31,
  'SEVEN': 32,
  'SMALL': 33,
  'SPEND': 34,
  'STAND': 35,
  'STRONG': 36,
  'THESE': 37,
  'THINK': 38,
  'TIMES': 39,
  'TODAY': 40,
  'TOMORROW': 41,
  'TRUST': 42,
  'UNTIL': 43,
  'VOTERS': 44,
  'WALES': 45,
  'WHERE': 46,
  'WOMEN': 47,
  'WOULD': 48,
  'YOUNG': 49
}

labels_from_key = defaultdict(list)
for k, v in label_dict.items():
    labels_from_key[v].append(k)

def predict_npz_dir(npz_path):
    path = pathlib.Path(npz_path)
    output_signature = (tf.TensorSpec(shape = (29, 1, 96, 96), dtype = tf.float32),
                        tf.TensorSpec(shape = (), dtype = tf.int16))
    predict_ds = tf.data.Dataset.from_generator(DataGenerator(path, set_type='test'),
                                         output_signature = output_signature)
    predict_ds = predict_ds.batch(1)

    predicted = model.predict(predict_ds)

    predicted_max = tf.argmax(predicted, axis=1)

    to_ret = []
    for idx, p in enumerate(predicted_max):
        pred = labels_from_key[p.numpy()][0]
        to_ret.append(pred)
        print(f"Word #{idx+1}: {pred} ({p.numpy()})")

    return to_ret

# FastAPI Server

In [8]:
from colabcode import ColabCode
import aiofiles
from fastapi import FastAPI, UploadFile, File, Request
from typing import List
import pathlib
import shutil

In [9]:
cc = ColabCode(port=12000, code=False)

In [10]:
app = FastAPI()

@app.get("/")
async def read_root():
  return {"about": "Lips Don't Lie Prediction Server"}


@app.post("/predict")
async def upload_files(files: List[UploadFile] = File(...)):
    pathlib.Path('/content/predict_ds/predict/test/').mkdir(parents=True, exist_ok=True)
    shutil.rmtree('/content/predict_ds/predict/test/')
    pathlib.Path('/content/predict_ds/predict/test/').mkdir(parents=True, exist_ok=True)

    for i, file in enumerate(files):
      async with aiofiles.open(f'/content/predict_ds/predict/test/{i}.npz', 'wb') as out_file:
        file_data = await file.read()
        await out_file.write(file_data)  # async write

    words = predict_npz_dir("/content/predict_ds")

    # Send the processed video or message back to the frontend
    return words

# Run Server

In [11]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://3d3f-35-193-211-220.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [600]
INFO:uvicorn.error:Started server process [600]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


3/3 [==============================] - 12s 204ms/step
Word #1: ACTION (0)
Word #2: CLOSE (7)
Word #3: WALES (45)
INFO:     85.65.155.82:0 - "POST /predict HTTP/1.1" 200 OK
3/3 [==============================] - 0s 82ms/step
Word #1: ACTION (0)
Word #2: CLOSE (7)
Word #3: WALES (45)
INFO:     85.65.155.82:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [600]
INFO:uvicorn.error:Finished server process [600]
